In [ ]:
import pyannote.core
from pydub import AudioSegment
file = {'uri':'1043_meal', 'audio':'/Users/andrei-macpro/Documents/Data/Audio/1043_meal.wav'}
import torch
# speech activity detection model trained on AMI training set
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami')
# obtain raw SAD scores (as `pyannote.core.SlidingWindowFeature` instance)
sad_scores = sad(file)

In [ ]:
from pyannote.audio.utils.signal import Binarize
binarize = Binarize(offset=0.52, onset=0.52, log_scale=True, 
                    min_duration_off=0.1, min_duration_on=0.1)

# speech regions (as `pyannote.core.Timeline` instance)
speech = binarize.apply(sad_scores, dimension=1)

In [ ]:
# convert timestamps to miliseconds 
type(speech[0])
# speech is a timeline made of segments

In [ ]:
#extract speech and non-speech segments (start and end time) as a dictionary
timestamps_speech=dict(speech)
timestamps_silence = dict(speech.gaps())

In [ ]:
file=AudioSegment.from_wav('/Users/andrei-macpro/Documents/Data/Audio/1043_meal.wav')

In [ ]:
# pydub works in miliseconds so we're going to convert the timestamps from the pyannote seconds to miliseconds
start_times_speech=[x*1000 for x in list(timestamps_speech.keys())]
end_times_speech=[x*1000 for x in list(timestamps_speech.values())]
start_times_silence=[x*1000 for x in list(timestamps_silence.keys())]
end_times_silence=[x*1000 for x in list(timestamps_silence.values())]

In [ ]:
speech_segments=[file[start_time:end_time] for start_time,end_time in zip(start_times_speech, end_times_speech)]
silence_segments=[file[start_time:end_time] for start_time,end_time in zip(start_times_silence, end_times_silence)]

In [ ]:
# export the concatenated speech and silence segments to disk
sum(speech_segments).export('/Users/andrei-macpro/Documents/Data/Audio/speech.wav', format="wav")
sum(silence_segments).export('/Users/andrei-macpro/Documents/Data/Audio/non-speech.wav', format="wav")

In [ ]:
# need to write a function that does everything sequentially while saving in memory only the result of binarization 
# and not the actual sliding window feature
# what we're going to need:
from pyannote.audio.utils.signal import Binarize
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami') # initialize the model; this isn't taking up too much memory
binarize = Binarize(offset=0.52, onset=0.52, log_scale=True, 
                    min_duration_off=0.1, min_duration_on=0.1)

# iterate through a list of speech/non-speech timelines that can store the binarized scores for each file 


In [ ]:
cd /Users/andrei-macpro/Documents/Data/Audio/Audio/Meal


In [ ]:
import os
path = '/Users/andrei-macpro/Documents/Data/Audio/Audio/Meal'  #if string starts with slash it is considered absolute
dirs = os.listdir( path )
file_names=sorted([i for i in os.listdir(".") if not i.startswith(".")])
file_names = [x.replace('.wav','') for x in file_names]

In [ ]:
# make a function to only return the timeline not the sliding window 
def timeline(file_name):
    speech=binarize.apply(sad(file_name),dimension=1)
    return speech

In [ ]:
import torch
sad = torch.hub.load('pyannote/pyannote-audio', 'sad_ami') # ok so this isn't taking up too much memory 

In [ ]:
timeline(file)

In [ ]:
def timeline(file_name):
    sad_scores=sad(file_name)
    speech=binarize.apply(sad_scores,dimension=1)
    del sad_scores
    return speech

In [ ]:
timestamps=timeline(file)

In [ ]:
del sad_scores

In [ ]:
# Importing gc module 
import gc 
  
# Returns the number of 
# objects it has collected 
# and deallocated 
collected = gc.collect() 
  
# Prints Garbage collector  
# as 0 object 
print("Garbage collector: collected", 
          "%d objects." % collected) 

In [ ]:
import gc
foo = range(10000000)
del foo
gc.collect()

In [ ]:
cd /Users/andrei-macpro/Documents/Data/Audio/Audio/Meal

In [ ]:
# make a function that returns a list
def timeline(file_name):
    timelines=list()
    sad_scores=sad(file_name)
    speech=binarize.apply(sad_scores,dimension=1)
    del sad_scores
    timelines.append(speech)
    return timelines

In [ ]:
def timeline(file_names):
    trial_list=list()
    for file_name in file_names:
        sad_score=sad(file_name)
        speech=binarize.apply(sad_scores,dimension=1)
        del sad_scores
        trial_list.append(speech)
    return trial_list

In [ ]:
def timeline(file_names):
    temp_list1=list()
    temp_list2=list()
    for file_name in file_names:
        sad_scores=sad(file_name)
        speech=binarize.apply(sad_scores,dimension=1)
        del sad_scores
        temp_list1.append(file_name.get('audio'))
        temp_list2.append(speech)
    trial_dict=dict(zip(temp_list1,temp_list2))
    return trial_dict

In [ ]:
# make dictionary for files and filenames
list_of_dictionaries=list()
files = dict()
for file_name in file_names:
    list_of_dictionaries.append({'audio':str(file_name)})

In [ ]:
new_dict=timeline(list_of_dictionaries[:2])

In [ ]:
new_dict['1043_meal.wav'].duration()

In [ ]:
timestamps_speech=dict(new_dict['1047_meal.wav'])
timestamps_speech

In [ ]:
# make new dictionaries that can hold the speech of each file
dictionary1={}
for file,timeline in zip(file_names, new_dict):
    dictionary1= dictionary1.update({str(file):new_dict[str(file)]})

In [ ]:
type(dictionary1)

In [ ]:
new_dict1 = dict.fromkeys([file for file in file_names], new_dict.values()) 